In [1]:
import os
import pandas as pd
import numpy as np

PATH = '../data/gold/'
STOCKS = ['AAPL', 'TSLA', 'JNJ', 'PFE', 'MARK', 'XOM', 'SO', 'NVDA', 'JPM', 'COST', 'NFLX', 'GE', 'AMT', 'T', 'AMZN']

os.listdir(PATH)

['jnjbeta.csv',
 'bonds.csv',
 'tslabeta.csv',
 'pricevol_high.csv',
 'coststats.csv',
 'US10yr.csv',
 'pfebeta.csv',
 'volatility90d.csv',
 'jpmbeta.csv',
 'aaplbeta.csv',
 'gebeta.csv',
 'nvdabeta.csv']

In [3]:
df=pd.read_csv(PATH+'pricevol_high.csv')

categories = ['DATE', 'CLOSE', 'VOLUME', 'HIGH']
df.columns = pd.MultiIndex.from_product([STOCKS, categories], names=['Stock', 'Info'])
df_long = df.stack(level='Stock').reset_index()
df_long.rename(columns={'level_0': 'Original Index', 'Stock': 'ID'}, inplace=True)

pricing_df = df_long[['ID', 'DATE', 'CLOSE', 'VOLUME', 'HIGH']]
pricing_df

Info,ID,DATE,CLOSE,VOLUME,HIGH
0,AAPL,2024-05-03,183.3800,163224109.0,187.0000
1,AMT,2024-05-03,181.7400,3130327.0,184.8900
2,AMZN,2024-05-03,186.2100,39172004.0,187.8700
3,COST,2024-05-03,743.9000,2323300.0,747.4900
4,GE,2024-05-03,164.1100,3968626.0,165.3000
...,...,...,...,...,...
52780,NVDA,2010-05-05,3.6250,133095328.0,3.7175
52781,PFE,2010-05-05,16.2858,99299909.0,16.4374
52782,SO,2010-05-05,34.0200,7230218.0,34.8500
52783,T,2010-05-05,19.4506,33486397.0,19.7283


In [ ]:
COLS=['DATE','ID','CLOSE','VOLUME','HIGH','VOL_90D','BETA_90D','BETA_30D','BETA_9D','SMA_9D','SMA_21D','SMA_60D']